# Creating our own prediction model void of predictor variables from data

In [173]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(6,6)})
%matplotlib inline
import warnings
warnings.simplefilter("ignore")

In [174]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
df = pd.read_csv("/Users/tylerrosselli/Desktop/Analysis-on-COMPAS-data-master/data/compas-scores-two-years.csv")
df.shape
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [175]:
pd.set_option('max_columns',53)
df.head()

df = df.drop(['first','last','race','decile_score','c_offense_date',
         'r_offense_date','is_violent_recid','type_of_assessment',
         'decile_score.1','score_text','v_type_of_assessment',
         'v_decile_score',"v_score_text",'v_screening_date',
         'in_custody','out_custody','c_case_number','c_days_from_compas',
         'is_recid','r_case_number','r_days_from_arrest','r_jail_in','r_jail_out',
         'violent_recid','vr_case_number','vr_charge_degree','vr_offense_date',
         'screening_date','days_b_screening_arrest','c_arrest_date','name',
         'compas_screening_date','id','dob','c_jail_in','c_jail_out'],axis=1)
df.shape

(7214, 17)

In [176]:
nums = df.select_dtypes(np.number).fillna(0)

In [177]:
print(nums.shape)
nums.head()
nums['time_in']= nums['end']-nums['start']
nums.drop(['start','end'],axis=1)
nums.reset_index()
nums.tail(20)

(7214, 10)


,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,priors_count.1,start,end,event,two_year_recid,time_in
7194,30,0,0,0,2,2,1,351,1,1,350
7195,35,0,0,0,8,8,62,393,1,1,331
7196,43,0,0,0,0,0,0,938,0,0,938
7197,28,0,0,0,1,1,0,764,0,0,764
7198,32,0,0,0,4,4,0,653,0,0,653
7199,23,0,0,0,0,0,2,913,0,0,911
7200,20,0,0,0,0,0,4,22,1,1,18
7201,30,0,1,0,19,19,9,144,1,1,135
7202,26,0,0,0,0,0,173,322,0,0,149
7203,44,0,0,0,0,0,0,887,0,0,887


In [178]:
obj = df.select_dtypes(object)

print(obj.r_charge_degree.unique(),'\n')
obj = obj.drop(['r_charge_desc','c_charge_desc','vr_charge_desc'],axis=1)
obj

[nan '(F3)' '(M1)' '(F2)' '(M2)' '(MO3)' '(F1)' '(F6)' '(F7)' '(CO3)'
 '(F5)'] 



,sex,age_cat,c_charge_degree,r_charge_degree
0,Male,Greater than 45,F,NaN
1,Male,25 - 45,F,(F3)
2,Male,Less than 25,F,(M1)
3,Male,Less than 25,F,NaN
4,Male,25 - 45,F,NaN
...,...,...,...,...
7209,Male,Less than 25,F,NaN
7210,Male,Less than 25,F,NaN
7211,Male,Greater than 45,F,NaN
7212,Female,25 - 45,M,NaN


In [179]:
obj_binary = pd.get_dummies(obj,
                            dtype="int64",
                            drop_first = True
).reset_index()

In [180]:
obj_binary.shape

(7214, 14)

In [181]:
df_clean = pd.concat([obj_binary, nums], axis = 1)

In [182]:
df_clean.head()
df_clean = df_clean.drop(['age','start','end','event'],axis=1)

In [183]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [193]:
y = df_clean['two_year_recid']
X = df_clean.drop('two_year_recid',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)
X.drop('index',axis=1)

,sex_Male,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_M,r_charge_degree_(F1),r_charge_degree_(F2),r_charge_degree_(F3),r_charge_degree_(F5),r_charge_degree_(F6),r_charge_degree_(F7),r_charge_degree_(M1),r_charge_degree_(M2),r_charge_degree_(MO3),juv_fel_count,juv_misd_count,juv_other_count,priors_count,priors_count.1,time_in,predictions,predict_prob
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,327,0,0.890850
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,150,1,0.013569
2,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,4,4,63,1,0.015419
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1174,0,0.999721
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1102,0,0.999281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,859,0,0.998791
7210,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,789,0,0.998115
7211,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,808,0,0.996303
7212,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,3,754,0,0.992548


In [194]:
model = LogisticRegression()

In [195]:
from sklearn.model_selection import cross_val_score
cross_val_score(
    model, X, y, scoring = "accuracy", cv = 10
).mean()

0.9744939123485772

In [196]:
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [197]:
preditions = model.predict(X)
predct_prob  = model.predict_proba(X)
predct_prob = predct_prob[:,0]

In [198]:
df_clean['predictions'] = preditions

In [199]:
df_clean['predict_prob'] = predct_prob

df_clean.head(20)

,index,sex_Male,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_M,r_charge_degree_(F1),r_charge_degree_(F2),r_charge_degree_(F3),r_charge_degree_(F5),r_charge_degree_(F6),r_charge_degree_(F7),r_charge_degree_(M1),r_charge_degree_(M2),r_charge_degree_(MO3),juv_fel_count,juv_misd_count,juv_other_count,priors_count,priors_count.1,two_year_recid,time_in,predictions,predict_prob
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,327,0,0.984894
1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,150,1,0.016049
2,2,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,4,4,1,63,1,0.003493
3,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1174,0,0.999347
4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,1102,0,0.999422
5,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,852,0,0.999082
6,6,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,1,35,1,0.001448
7,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,265,0,0.967170
8,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,745,0,0.999476
9,9,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,428,1,0.031422


In [200]:
df_clean[df_clean['two_year_recid']!=df_clean['predictions']].shape

(158, 23)